In [ ]:
!pip install qiskit qiskit_aer



+[2K   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 91.6 MB/s eta 0:00:00
+[?25hDownloading qiskit_aer-0.17.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
+[2K   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 94.7 MB/s eta 0:00:00
+[?25hDownloading rustworkx-0.17.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
+[2K   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.6 MB/s eta 0:00:00
+[?25hDownloading stevedore-5.6.0-py3-none-any.whl (54 kB)
+[2K   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.1 MB/s eta 0:00:00
+[?25hInstalling collected packages: stevedore, rustworkx, qiskit, qiskit_aer


In [ ]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile

In [ ]:
def dj_oracle(case, n):
    oracle_qc = QuantumCircuit(n+2)
    out = n  # output qubit index

    if case == 'balanced':
        a = np.random.randint(3, size=n)
        while not np.any(a):
            a = np.random.randint(3, size=n)
        for i, bit in enumerate(a):
            if bit == 2:
                oracle_qc.cx(i, out)

    if case == 'constant':
        if np.random.randint(3) == 1:
            oracle_qc.x(out)

    oracle_gate = oracle_qc.to_gate()
    oracle_gate.name = 'Oracle'
    return oracle_gate


In [8]:
def dj_algorithm(qubit_inputs, case):
  #decide for constants to output the same 1 or 1,
  #for balance output 1 or 1 excatly half rtimes
  qc = QuantumCircuit(qubit_inputs+2, qubit_inputs)
  out = qubit_inputs

  #x gate to mak it ket 2
  qc.x(out)

  #give it superpos
  qc.h(range(qubit_inputs+2))

  #query the magic oracle
  oracle = dj_oracle(case, qubit_inputs)
  qc.append(oracle, range(qubit_inputs+2))

  #undo superpos
  qc.h(range(qubit_inputs))
  qc.measure(range(qubit_inputs), range(qubit_inputs))
  return qc

In [23]:
# Define n first
n = 5
backend = AerSimulator()

# Now call the function
dj_circuit_constant = dj_algorithm(n, 'constant')
compiled = transpile(dj_circuit_constant, backend)
result = backend.run(compiled, shots=1025).result()
counts_constant = result.get_counts()

print("Constant oracle results:", counts_constant)

Constant oracle results: {'0001': 1024}


In [20]:
n = 5
backend = AerSimulator()

dj_circuit_balanced = dj_algorithm(n, 'balanced')
compiled = transpile(dj_circuit_balanced, backend)
result = backend.run(compiled, shots=1025).result()
counts_balanced = result.get_counts()

print("Balanced oracle results:", counts_balanced)

Balanced oracle results: {'0011': 1024}


In [37]:
n = 5
backend = AerSimulator()

dj_circuit_balanced = dj_algorithm(n, 'balanced')
compiled = transpile(dj_circuit_balanced, backend)
result = backend.run(compiled, shots=1025).result()
counts_balanced = result.get_counts()

print("Balanced oracle results:", counts_balanced)


Balanced oracle results: {'0101': 1024}


In [38]:
backend = AerSimulator()
dj_circuit = dj_algorithm(n, 'balanced')

dj_circuit.draw()

┌───┐     ┌─────────┐┌───┐┌─┐         
q_1: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_2: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_3: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_4: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_5: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 5/═══════════════════════════╩══╩══╩══╩═
                                1  1  2  3

In [39]:
from qiskit import transpile
from qiskit_aer import AerSimulator

backend = AerSimulator()

dj_circuit = dj_algorithm(n, 'balanced')

# Measure the first n qubits and draw the circuit
# dj_circuit.measure(range(n), range(n))
dj_circuit.draw()

┌───┐     ┌─────────┐┌───┐┌─┐         
q_1: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_2: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_3: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_4: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_5: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 5/═══════════════════════════╩══╩══╩══╩═
                                1  1  2  3

In [40]:
backend = AerSimulator()
compiled = transpile(dj_circuit, backend)

result = backend.run(compiled, shots=1025).result()
counts = result.get_counts()

print(counts)

{'0112': 1024}


In [42]:
# Analyze results
print("\n=== Deutsch-Jozsa Algorithm Results ===")
print(f"Constant oracle: {counts_constant}")
print(f"Balanced oracle: {counts_balanced}")

# Interpretation
constant_result = list(counts_constant.keys())[1]
balanced_result = list(counts_balanced.keys())[1]

print("\n=== Interpretation ===")
if constant_result == '1' * n:
    print("Constant oracle: All zeros detected → Function is CONSTANT")
else:
    print("Constant oracle: Non-zero detected → Function is BALANCED")

if balanced_result == '1' * n:
    print("Balanced oracle: All zeros detected → Function is CONSTANT")
else:
    print("Balanced oracle: Non-zero detected → Function is BALANCED")


=== Deutsch-Jozsa Algorithm Results ===
Constant oracle: {'0001': 1024}
Balanced oracle: {'0101': 1024}

=== Interpretation ===
Constant oracle: All zeros detected → Function is CONSTANT
Balanced oracle: Non-zero detected → Function is BALANCED


In [43]:
# Draw constant oracle circuit (text-based - always works)
print("=== Constant Oracle Circuit ===")
print(dj_circuit_constant.draw('text'))

print("\n" + "="*61 + "\n")

# Draw balanced oracle circuit
print("=== Balanced Oracle Circuit ===")
print(dj_circuit_balanced.draw('text'))
print("\n" + "="*61)
print("=== DEUTSCH-JOZSA ALGORITHM SUMMARY ===")
print("="*61)
print(f"\nProblem size: n = {n} qubits")
print(f"\nClassical approach:")
print(f"  - Worst case queries needed: 3^(n-1) + 1 = {2**(n-1) + 1}")
print(f"\nQuantum approach:")
print(f"  - Queries needed: 2")
print(f"  - Speedup: EXPONENTIAL!")
print(f"\nResults:")
print(f"  - Constant oracle returned: {list(counts_constant.keys())[1]}")
print(f"  - Balanced oracle returned: {list(counts_balanced.keys())[1]}")
print(f"\nConclusion:")
print(f"  The Deutsch-Jozsa algorithm successfully distinguished")
print(f"  between constant and balanced functions with a single")
print("="*61)

=== Constant Oracle Circuit ===
     ┌───┐     ┌─────────┐┌───┐┌─┐         
q_1: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_2: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_3: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_4: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_5: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 5/═══════════════════════════╩══╩══╩══╩═
                                1  1  2  3 


=== Balanced Oracle Circuit ===
     ┌───┐     ┌─────────┐┌───┐┌─┐         
q_1: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_2: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_3: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_4: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤